In [1]:
import pandas as pd

from scripts.preprocessing import preprocess_pipeline
from scripts.model import XGBOOSTPredictor

In [2]:
df = pd.read_csv("./bases/PEDE_2022.csv")

In [3]:
df.head()

,RA,Turma,Nome,Ano_Nascimento,Idade,Gênero,Ano ingresso,Instituição de ensino,Pedra,INDE,...,IDA,Indicado,Atingiu PV,IPV,IAN,Defasagem,Destaque IEG,Destaque IDA,Destaque IPV,Ano_Coleta_Dados
0,RA-1,A,Aluno-1,2003,19,Menina,2016,Escola Pública,Quartzo,"5,783",...,"4,0",Sim,Não,"7,278","5,000",-1,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...,2022
1,RA-2,A,Aluno-2,2005,17,Menina,2017,Rede Decisão,Ametista,"7,055",...,"6,8",Não,Não,"6,778","10,000",0,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...,2022
2,RA-3,A,Aluno-3,2005,17,Menina,2016,Rede Decisão,Ágata,"6,591",...,"5,6",Não,Não,"7,556","10,000",0,Destaque: A sua boa entrega das lições de casa.,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Destaque: A sua boa integração aos Princípios ...,2022
3,RA-4,A,Aluno-4,2005,17,Menino,2017,Rede Decisão,Quartzo,"5,951",...,"5,0",Não,Não,"5,278","10,000",0,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...,2022
4,RA-5,A,Aluno-5,2005,17,Menina,2016,Rede Decisão,Ametista,"7,427",...,"5,2",Não,Não,"7,389","10,000",0,Destaque: A sua boa entrega das lições de casa.,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...,2022


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   RA                     860 non-null    object
 1   Turma                  860 non-null    object
 2   Nome                   860 non-null    object
 3   Ano_Nascimento         860 non-null    int64 
 4   Idade                  860 non-null    int64 
 5   Gênero                 860 non-null    object
 6   Ano ingresso           860 non-null    int64 
 7   Instituição de ensino  860 non-null    object
 8   Pedra                  860 non-null    object
 9   INDE                   860 non-null    object
 10  Cg                     860 non-null    object
 11  Cf                     860 non-null    int64 
 12  Ct                     860 non-null    int64 
 13  Numero_Avaliacoes      860 non-null    int64 
 14  Rec Av1                860 non-null    object
 15  Rec Av2                

In [5]:
X_train, X_test, y_train, y_test = preprocess_pipeline(df)

In [6]:
X_train.head()

,Ano_Nascimento,Idade,Ano ingresso,INDE,Ranking_Geral,Ranking_Na_Fase,Ranking_Na_Turma,Numero_Avaliacoes,Indicador_Auto_Avaliacao,Indicador_Engajamento,...,Indicador_Aprendizagem,Indicador_Adequacao_Nivel,Defasagem,Ano_Coleta_Dados,Destaque IEG_num,Destaque IDA_num,Destaque IPV_num,Instituição de ensino_Escola JP II,Instituição de ensino_Escola Pública,Instituição de ensino_Rede Decisão
51,2009,13,2018,7.861,183.0,12,5,3,9.6,8.9,...,6.2,10.0,2,2022,1,-1,1,0.0,0.0,1.0
383,2010,12,2017,7.558,291.0,44,2,3,9.0,8.4,...,8.1,5.0,-1,2022,1,1,1,0.0,1.0,0.0
310,2006,16,2022,5.652,771.0,114,13,3,7.9,5.6,...,5.2,2.5,-3,2022,-1,-1,-1,0.0,1.0,0.0
630,2010,12,2022,6.928,515.0,133,7,3,8.5,6.8,...,7.0,5.0,-2,2022,-1,-1,-1,0.0,1.0,0.0
559,2011,11,2020,4.949,826.0,189,11,2,6.9,7.2,...,3.8,5.0,-1,2022,-1,-1,-1,0.0,1.0,0.0


In [9]:
y_train

51     0
383    0
310    0
630    0
559    0
      ..
304    0
807    0
185    0
479    1
124    0
Name: Atingiu PV, Length: 688, dtype: int64

In [10]:
model = XGBOOSTPredictor()

model.fit(X_train, y_train)

Modelo treinado com sucesso!


In [13]:
model.save("./models/xgboost_model.pkl")

Modelo salvo em ./models/xgboost_model.pkl
